In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import json
import os
import psycopg2

In [2]:
def get_connection():
    conn = {}
    with open('../db_connection/db_config.json') as my_file:
        db_conf = json.loadad(my_file)
    conn_str = "host={} dbname={} user={} password={}".format(
                db_conf['host'], db_conf['database'], 
                db_conf['user'], db_conf['passw'])
    try:
        conn = psycopg2.connect(conn_str)
        return conn
    except:
        return 'error in connecting to database'

In [3]:
get_connection()

<connection object at 0x7fb460df3b48; dsn: 'host=41.89.94.68 dbname=mohdsl user=boa_user password=xxxxxxxxxxxxxxxxx', closed: 0>

In [28]:
#get a specific facility
def get_facility_by_id(facility_id):
    conn = get_connection()
    all_facilities = pd.DataFrame()
    query = "SELECT * FROM facilities_facility WHERE id = '%s' ;" %(facility_id)    
    for chunk in pd.read_sql(query, con=conn, chunksize=10000):
        all_facilities = all_facilities.append(chunk)

    
    return all_facilities

In [29]:
get_facility_by_id('2e21b2de-7c5b-4db7-bb1d-26930af9274d')

,id,deleted,active,name,official_name,code,registration_number,abbreviation,description,number_of_beds,...,_effectivefrom,_effectiveto,ward_id,facility_type_id,keph_level_id,owner_id,parent_id,sub_county_id,town_id,facility_sk
0,2e21b2de-7c5b-4db7-bb1d-26930af9274d,False,True,Kenyatta National Hospital,Kenyatta National Hospital,13023,None,None,None,1455,...,1900-01-01,2199-12-31,e86bec61-9d3f-4825-b23a-87219e8ecce4,aa32be23-693e-45fb-bc3f-0b24d6190de3,169f6176-8166-4d51-9b7c-e5d5fee1e9a1,1b0b8814-a3d0-4e76-bfc9-4dea62a58b60,None,None,4763f1a1-aed4-4678-a1b2-0798935f4cf6,1471


In [30]:
#get a specific county
def get_organization_units():
    conn = get_connection()
    all_orgunits = pd.DataFrame()
    for chunk in pd.read_sql('SELECT * FROM dim_dhis_organisationunit', con=conn, chunksize=10000):
        all_orgunits = all_orgunits.append(chunk)
    
    return all_orgunits

In [31]:
get_organization_units()

,organisationunitid,organisationunitname,mflcode,parentid,shortname,code,openingdate,closeddate,coordinates,latitude,...,email,phonenumber,uid,description,uuid,_datecreated,_eventexecutionkey,_lastaction,_mergekey,_sourcesystemcode
0,512884,HABASWEIN DISTRICT HOSPITAL,NaN,22694.0,HABASWEIN DISTRICT HOSPITAL,None,1900-01-01,None,None,None,...,None,None,lDSUtXy5gIH,None,6e651fcf-b8b7-4a05-9b62-fb55a1a138ad,2016-02-01,None,I,None,None
1,513316,Bomeni Community Unit,NaN,24743.0,Bomeni Community Unit,None,2013-10-01,None,None,None,...,None,0721-499562,IbIzxMl0Nox,None,365a68bc-017f-4073-8f3f-5fbe8603f06d,2016-02-01,None,I,None,None
2,423684,Sirisia Community Unit,NaN,44829.0,Sirisia Community Unit,None,2013-06-01,None,None,None,...,None,0712002818,FRVuojduxDT,None,060dbd69-6c10-419a-a511-1aada53f046e,2016-02-01,None,I,None,None
3,512885,HABASWEIN DISTRICT HOSPITAL,NaN,22694.0,HABASWEIN DISTRICT HOSPITAL,None,1900-01-01,None,None,None,...,None,None,mUJamp16ulT,None,f367c361-ff80-48b0-b6a6-bbffd8a3329a,2016-02-01,None,I,None,None
4,7713821,mwakini dispensary,NaN,70091.0,mwakini dispensary,None,2015-04-27,None,"[37.73762,-1.23056]",None,...,None,None,JeZwi7ZFngx,None,62ce9b69-5751-4897-8811-67ff93ba1aed,2016-02-01,None,I,None,None
5,6529959,Kajaja 2,NaN,274788.0,Kajaja 2,None,2015-03-10,None,None,None,...,None,None,vbYdPMs0iXI,None,71cb365c-2326-46d5-979f-9dc94797876d,2016-02-01,None,I,None,None
6,8445672,DR.WALUMBE,20550.0,126.0,DR.WALUMBE,20550,2015-06-26,None,None,None,...,None,None,JCWEmzlEsUd,None,103f40f9-f539-4507-b1cd-8137d7545c28,2016-02-01,None,I,None,None
7,7723507,Gumarey Community Health Unit,NaN,22695.0,Gumarey Community Health Unit,None,2015-03-01,None,None,None,...,None,0712105080,SPVRRIHvbIe,None,6bddde7b-11c6-47a8-bbc8-6b6a333d5693,2016-02-01,None,I,None,None
8,254980,NON EXISTENT,NaN,44277.0,NON EXISTENT,None,1899-12-31,None,None,None,...,None,None,PIdRPNJNPGu,None,8e396656-cdc3-48a9-9898-a18f13578557,2016-02-01,None,I,None,None
9,62222,National Youth Service Turbo Dispensary,NaN,95.0,NYS Dispensary,None,1899-12-31,None,None,None,...,None,0713594880,UOKEv02h8eL,None,43ea519c-8b2a-4084-b7e2-059efd61275f,2016-02-01,None,I,None,None


In [32]:
def get_facility_org_units(facility_id):
    #get the facility    
    facility = get_facility_by_id(facility_id)
    facility['code'] = facility['code'].astype(str)
    
    #get all org units
    all_orgunits = get_organization_units()
    all_orgunits['code'] = all_orgunits['code'].astype(str)
    facility_orgunits = pd.merge(all_orgunits, facility, on='code')
    
    return facility_orgunits

In [33]:
get_facility_org_units('2e21b2de-7c5b-4db7-bb1d-26930af9274d')

,organisationunitid,organisationunitname,mflcode,parentid,shortname,code,openingdate,closeddate,coordinates,latitude,...,_effectivefrom,_effectiveto,ward_id,facility_type_id,keph_level_id,owner_id,parent_id,sub_county_id,town_id,facility_sk
0,33707,Kenyatta National Hospital,13023.0,6458775.0,KNH,13023,1990-01-01,None,"[36.80734,-1.30088]",None,...,1900-01-01,2199-12-31,e86bec61-9d3f-4825-b23a-87219e8ecce4,aa32be23-693e-45fb-bc3f-0b24d6190de3,169f6176-8166-4d51-9b7c-e5d5fee1e9a1,1b0b8814-a3d0-4e76-bfc9-4dea62a58b60,None,None,4763f1a1-aed4-4678-a1b2-0798935f4cf6,1471


In [6]:
def get_data_element_by_id(data_element_id):
    conn = get_connection()
    all_data_elements = pd.DataFrame()
    query = "SELECT * FROM dim_dhis_dataelement WHERE dataelementid = %s ;" %(data_element_id)
    for chunk in pd.read_sql(query, con=conn, chunksize=10000):
        all_data_elements = all_data_elements.append(chunk)

    
    return all_data_elements

In [7]:
get_data_element_by_id(164968)

,dataelementid,dataelementname,shortname,code,description,valuetype,domaintype,aggregationtype,zeroissignificant,numbertype,uid,_datecreated,_eventexecutionkey,_lastaction,_mergekey,_sourcesystemcode
0,164968,Normal Weight for Age 6-23 months,Normal Weight 6-23 months,None,None,int,AGGREGATE,sum,False,number,ikI4r8YuZZ0,None,None,None,None,None


In [36]:
def get_facility_dataelement_datavalues(dataelement_id, facility_id):
    conn = get_connection()
    orgnaization_unit = get_facility_org_units(facility_id)
    dataelement = get_data_element_by_id(dataelement_id)
    all_datavalues = DataFrame()
    
    query = "SELECT * FROM fact_dhis_datavalue WHERE dataelementid = %s AND sourceid = %s ;" %(dataelement_id, orgnaization_unit['organisationunitid'][0])
    for chunk in pd.read_sql(query, con=conn , chunksize=10000):
        all_datavalues = all_datavalues.append(chunk)
    
    
    return all_datavalues

In [37]:
values = get_facility_dataelement_datavalues(5154416, 'ab3ad01b-9716-4f7c-a4d7-5f69b12b44cb')

In [14]:
values

,dataelementid,periodid,sourceid,categoryoptioncomboid,attributeoptioncomboid,value,storedby,lastupdated,comment,followup,created,_datecreated,_eventexecutionkey,_lastaction,_mergekey,_sourcesystemcode
0,5154416,2505073,24708,5154243,15,59,Tsigo,2015-05-08 16:02:11.170,None,False,2015-05-08 16:02:11.171,2015-12-17,None,None,None,None
1,5154416,2505073,24708,5154302,15,923,Tsigo,2015-05-08 16:01:51.330,None,False,2015-05-08 16:01:51.331,2015-12-17,None,None,None,None
2,5154416,2506122,24708,5154243,15,215,Tsigo,2015-04-13 11:32:37.746,None,False,2015-04-13 11:32:37.747,2015-12-17,None,None,None,None
3,5154416,2506122,24708,5154302,15,942,Tsigo,2015-04-13 11:32:19.001,None,False,2015-04-13 11:32:19.003,2015-12-17,None,None,None,None


In [14]:
def get_facility_dataelement_datavalues2(dataelement_id, facility_id, period_id):
    conn = get_connection()
    orgnaization_unit = get_facility_org_units(facility_id)
    dataelement = get_data_element_by_id(dataelement_id)
    all_datavalues = DataFrame()
    
    query = "SELECT * FROM fact_dhis_datavalue WHERE dataelementid = %s AND sourceid = %s AND periodid = %s ;" %(dataelement_id, orgnaization_unit['organisationunitid'][0], period_id)
    for chunk in pd.read_sql(query, con=conn , chunksize=10000):
        all_datavalues = all_datavalues.append(chunk)
    
    
    return all_datavalues

In [15]:
get_facility_dataelement_datavalues2(5154416, 'ab3ad01b-9716-4f7c-a4d7-5f69b12b44cb', 2505073)

,dataelementid,periodid,sourceid,categoryoptioncomboid,attributeoptioncomboid,value,storedby,lastupdated,comment,followup,created,_datecreated,_eventexecutionkey,_lastaction,_mergekey,_sourcesystemcode
0,5154416,2505073,24708,5154243,15,59,Tsigo,2015-05-08 16:02:11.170,None,False,2015-05-08 16:02:11.171,2015-12-17,None,None,None,None
1,5154416,2505073,24708,5154302,15,923,Tsigo,2015-05-08 16:01:51.330,None,False,2015-05-08 16:01:51.331,2015-12-17,None,None,None,None


In [ ]:
def get_indicator_datavalues(facility_id)